<a href="https://colab.research.google.com/github/emrapport/w266-final-project/blob/master/modeling/no_polit_names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Remove names of politicians.



In [0]:
# this notebook is based off of this blog post: 
# https://realpython.com/python-keras-text-classification/#reader-comments

In [0]:
DATASET_NAME = "no_politicians"
MODEL_NAME = "no_politicians"
MAX_SEQ_LENGTH = 20
TRAINING_SET_SIZE = 4000000
VAL_SET_SIZE = 2000000
NUM_EPOCHS = 30
BATCH_SIZE = 1000
MAX_SEQUENCE_LENGTH = 20


# I don't think we have a good reason to do this right now
# but we might eventually 
SAVE_TOKENIZED_DATA = False

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import hstack, vstack
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential, layers
from keras.utils import plot_model
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

Using TensorFlow backend.


     |████████████████████████████████| 51kB 2.0MB/s 
  Created wheel for gcsfs: filename=gcsfs-0.3.1-py2.py3-none-any.whl size=17936 sha256=cc2640790da74298116090025fe3d5490f03fe3ad2a700696fb0b29ed40e2acb
  Stored in directory: /root/.cache/pip/wheels/9d/2b/6f/86954f0d8caa1173841e62bb780dc0f8693bd268e04a267682
Successfully built gcsfs


In [3]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Updated property [core/project].


In [4]:
train_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/train.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
dev_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/dev.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
len(train_df)

9879016

In [0]:
len(dev_df)

2292907

In [6]:
# Get a list of the politicians' names.
# I am using just the names listed in the op_name series.
polit_names = train_df['op_name']
polit_names.drop_duplicates(inplace=True)
full_names_list = polit_names.to_list()
polit_names_list = [name for names in full_names_list for name in names.split()]
polit_names_list

['Roger',
 'Williams',
 'Tom',
 'Carper',
 'Jared',
 'Polis',
 'James',
 'Lankford',
 'Tulsi',
 'Gabbard',
 'Ted',
 'Yoho',
 'Rand',
 'Paul',
 'Luke',
 'Messer',
 'Kathy',
 'Castor',
 'Lou',
 'Barletta',
 'Rob',
 'Bishop',
 'Kay',
 'Granger',
 'Nancy',
 'Pelosi',
 'Susan',
 'Collins',
 'John',
 'Faso',
 'Jeanne',
 'Shaheen',
 'Daniel',
 'Donovan',
 'John',
 'Yarmuth',
 'Jeff',
 'Denham',
 'Diane',
 'Black',
 'Jim',
 'Himes',
 'Roy',
 'Blunt',
 'David',
 'Scott',
 'Zoe',
 'Lofgren',
 'Angus',
 'King',
 'Michelle',
 'Lujan',
 'Grisham',
 'Collin',
 'Peterson',
 'Gregg',
 'Harper',
 'Tom',
 'Graves',
 'Mark',
 'Amodei',
 'Brad',
 'Sherman',
 'Jodey',
 'Arrington',
 'Dave',
 'Reichert',
 'Norma',
 'Torres',
 'Al',
 'Franken',
 'Brad',
 'Wenstrup',
 'Peter',
 'Roskam',
 'Dina',
 'Titus',
 'Lisa',
 'Blunt',
 'Rochester',
 'David',
 'Rouzer',
 'Elijah',
 'Cummings',
 'Jeff',
 'Fortenberry',
 'Katherine',
 'Clark',
 'Dutch',
 'Ruppersberger',
 'David',
 'McKinley',
 'Erik',
 'Paulsen',
 'Pat',

In [0]:
len(polit_names_list)

468

In [0]:
# Dedup the list again now that it's split.
new_list = []
for name in polit_names_list:
    if name not in new_list:
        new_list.append(name)
polit_names_list = new_list

In [8]:
num_responses_per_post = train_df.post_id.value_counts().reset_index()
num_responses_per_post.columns = ['post_id', 'num_responses']
too_big_posts = num_responses_per_post[num_responses_per_post.num_responses > 50]
posts_to_sample = too_big_posts.post_id.values
# this gets all the rows for posts we DON'T need to sample 
new_train_df = train_df[~train_df.post_id.isin(posts_to_sample)]
# this should be true
assert(len(too_big_posts) + new_train_df.post_id.nunique() == train_df.post_id.nunique())
too_big_post_rows = train_df[train_df.post_id.isin(posts_to_sample)]
sampled_rows = too_big_post_rows.groupby('post_id').apply(lambda x: x.sample(n=50)).reset_index(drop=True)
new_train_df = pd.concat([new_train_df, sampled_rows])
new_train_df.shape

(3962284, 8)

In [0]:
start_time = time.time()
for name in polit_names_list:
    name = name.lower()
    new_train_df['response_text'] = new_train_df['response_text'].str.replace(name, 'PolitName')
    print("replaced:",name)
print("minutes to complete:",(time.time()-start_time)/60)

replaced: roger
replaced: williams
replaced: tom
replaced: carper
replaced: jared
replaced: polis
replaced: james
replaced: lankford
replaced: tulsi
replaced: gabbard
replaced: ted
replaced: yoho
replaced: rand
replaced: paul
replaced: luke
replaced: messer
replaced: kathy
replaced: castor
replaced: lou
replaced: barletta
replaced: rob
replaced: bishop
replaced: kay
replaced: granger
replaced: nancy
replaced: pelosi
replaced: susan
replaced: collins
replaced: john
replaced: faso
replaced: jeanne
replaced: shaheen
replaced: daniel
replaced: donovan
replaced: yarmuth
replaced: jeff
replaced: denham
replaced: diane
replaced: black
replaced: jim
replaced: himes
replaced: roy
replaced: blunt
replaced: david
replaced: scott
replaced: zoe
replaced: lofgren
replaced: angus
replaced: king
replaced: michelle
replaced: lujan
replaced: grisham
replaced: collin
replaced: peterson
replaced: gregg
replaced: harper
replaced: graves
replaced: mark
replaced: amodei
replaced: brad
replaced: sherman
repla

In [0]:
start_time = time.time()
for name in polit_names_list:
    name = name.lower()
    dev_df['response_text'] = dev_df['response_text'].str.replace(name, 'PolitName')
    print("replaced:",name)
print("minutes to complete:",(time.time()-start_time)/60)

replaced: roger
replaced: williams
replaced: tom
replaced: carper
replaced: jared
replaced: polis
replaced: james
replaced: lankford
replaced: tulsi
replaced: gabbard
replaced: ted
replaced: yoho
replaced: rand
replaced: paul
replaced: luke
replaced: messer
replaced: kathy
replaced: castor
replaced: lou
replaced: barletta
replaced: rob
replaced: bishop
replaced: kay
replaced: granger
replaced: nancy
replaced: pelosi
replaced: susan
replaced: collins
replaced: john
replaced: faso
replaced: jeanne
replaced: shaheen
replaced: daniel
replaced: donovan
replaced: yarmuth
replaced: jeff
replaced: denham
replaced: diane
replaced: black
replaced: jim
replaced: himes
replaced: roy
replaced: blunt


In [0]:
start_time = time.time()
# Remove 1 and 2 word responses from training set.
# Split response string by spaces.
new_train_df['split_response'] = new_train_df['response_text'].str.split()
# Remove the split responses that have length 1 or 2.
new_train_df = new_train_df[new_train_df['split_response'].str.len() > 2]
new_train_df.head()
print("minutes to complete:",(time.time()-start_time)/60)

minutes to complete: 1.3340154806772868


In [0]:
# def get_short_response(series_list,length=20):
#   '''Take a list of strings and a goal max length. Return the goal length list
#   taken half from the beginning of the orginal list and half from the end of 
#   the original list.'''
#   if len(series_list) > length:
#     new_list = series_list[:length//2] + series_list[-length//2:]
#     return new_list
#   else:
#     return series_list

In [0]:
# train_df['short_response'] = train_df['split_response'].apply(get_short_response)

In [0]:
# Remove 1 and 2 word responses from dev set.
# Split response string by spaces.
dev_df['split_response'] = dev_df['response_text'].str.split()
# Remove the split responses that have length 1 or 2.
dev_df = dev_df[dev_df['split_response'].str.len() > 2]
len(dev_df)

2041945

In [0]:
# Shorten the lists to the max length (using 20 for now)
# dev_df['short_response'] = dev_df['split_response'].apply(get_short_response)

In [0]:
# Turn the list back into one string.
# dev_df['response_text'] = dev_df['short_response'].str.join(' ')

In [0]:
# Turn the list back into one string.
# train_df['response_text'] = train_df['short_response'].str.join(' ')

In [0]:
# Remove the columns of lists.
train_df = train_df.drop(['split_response'], axis=1)
dev_df = dev_df.drop(['split_response'], axis=1)


In [0]:
dev_df.head()

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category
0,1965080,86681682,M,29909,James,thanks for passing this ridiculous legislation. https://www.youtube.com/watch?v=-y4wd8royre,Jim Banks,Congress_Republican
1,1965081,86681682,M,29909,Martha,handsome young man on the left. 😂 mom,Jim Banks,Congress_Republican
2,1965082,86681682,M,29911,Preston,you are sideways!,Jim Banks,Congress_Republican
3,1965083,86681682,M,29911,Bobby,hi congressman banks,Jim Banks,Congress_Republican
4,1965084,86681682,M,29911,Landon,"jim, i love your office. the 5th floor is for winners.",Jim Banks,Congress_Republican


In [0]:
import gc
gc.collect()

49

In [0]:
len(train_df)

8685575

In [0]:
# Set seeds for reproducible results.
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [0]:
# shuffle the data
# be sure to do this before you extract X's and y's!!
train_df = new_train_df.sample(frac=1)
dev_df = dev_df.sample(frac=1)

In [0]:
train_df.op_gender.value_counts()

M    2900753
W    1061531
Name: op_gender, dtype: int64

In [0]:
dev_df.op_gender.value_counts()

M    1731288
W     310657
Name: op_gender, dtype: int64

In [0]:
# Set the y values.
y_train = train_df.op_gender.values
y_dev = dev_df.op_gender.values

In [0]:
# Turn y values into 0 (female) and 1 (male).
def turn_to_ints(li):
    final_list = []
    for gender in li:
        if gender=='M':
            final_list.append(1)
        else:
            final_list.append(0)
    return final_list
            
y_train = turn_to_ints(y_train)
y_dev = turn_to_ints(y_dev)

In [0]:
# Convert y values to numpy arrays.
y_train = np.asarray(y_train)
y_dev = np.asarray(y_dev)

In [0]:
def get_text_list(init_list):
    sentences = []
    for sentence in init_list:
        if type(sentence) != str:
            sentences.append("")
        else:
            sentences.append(sentence)
    return sentences

new_sentences_train = get_text_list(train_df.response_text.values) 
new_sentences_test = get_text_list(dev_df.response_text.values)

In [0]:
# Tokenize the sentences.
time_start = time.time()

# this is the default list of filters + apostrophe
# added because we have dealt with common contractions, so other apostrophes should mostly be possessive 
tokenizer = Tokenizer(num_words=200000, filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(new_sentences_train)

X_train = tokenizer.texts_to_sequences(new_sentences_train)
X_test = tokenizer.texts_to_sequences(new_sentences_test)

vocab_size = len(tokenizer.word_index) + 1

currentTime = time.gmtime(time.time() - time_start)

#Convert the gmtime struct to a string
timeStr = time.strftime("%M minutes, %S seconds", currentTime)

print("Tokenized in {}".format(timeStr))

Tokenized in 05 minutes, 27 seconds


In [0]:
# Pad the tokenized data so all sequences are the same length; see first cell for max length.
# Length should be the same as in shortened response length above...
from keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, padding='post', maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, padding='post', maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
if SAVE_TOKENIZED_DATA:
  x_train_path = 'X_train_{}.pkl'.format(DATASET_NAME)
  x_dev_path = 'X_dev_{}.pkl'.format(DATASET_NAME)
  y_train_path = 'y_train_{}.pkl'.format(DATASET_NAME)
  y_dev_path = 'y_dev_{}.pkl'.format(DATASET_NAME)

  with open(x_train_path, 'wb') as file:
      pickle.dump(X_train, file)   
  with open(x_dev_path, 'wb') as file:
      pickle.dump(X_test, file)
  with open(y_train_path, 'wb') as file:
      pickle.dump(y_train, file)
  with open(y_dev_path, 'wb') as file:
      pickle.dump(y_dev, file)

  # copy to bucket
  !gsutil cp /content/{x_train_path} gs://fb-congressional-data/test
  !gsutil cp /content/{x_dev_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_train_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_dev_path} gs://fb-congressional-data/

In [0]:
# Get the glove embeddings.
!gsutil cp gs://fb-congressional-data/glove* /tmp/

Copying gs://fb-congressional-data/glove.6B.100d.txt...
Copying gs://fb-congressional-data/glove.6B.200d.txt...
Copying gs://fb-congressional-data/glove.6B.300d.txt...
Copying gs://fb-congressional-data/glove.6B.50d.txt...
- [4 files][  2.1 GiB/  2.1 GiB]   43.7 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://fb-congressional-data/glove.6B.zip...
\ [5 files][  2.9 GiB/  2.9 GiB]   17.5 MiB/s                                   
Operation completed over 5 objects/2.9 GiB.                                      


In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_dim = 100
embedding_matrix = create_embedding_matrix(
                     '/tmp/glove.6B.{}d.txt'.format(embedding_dim),
                      tokenizer.word_index, embedding_dim)

In [0]:
# trying to figure out which words are empty here
# counter = 0
# empty_indexes = []
# for index, row in enumerate(embedding_matrix):
#   if sum(row) == 0:
#     empty_indexes.append(index)
#     counter += 1
#   if counter > 1000:
#     break

# for idx in empty_indexes:
#   try:
#     print(tokenizer.index_word[idx])
#   except:
#     print("No entry for {}".format(idx))


In [0]:
# hmmmm....
# nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
# nonzero_elements / vocab_size

0.15381079935942088

In [0]:
embedding_matrix.shape

(597742, 100)

In [0]:
# Initialize the model.
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=MAX_SEQUENCE_LENGTH, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 100)           59774200  
_________________________________________________________________
conv1d (Conv1D)              (None, 16, 128)           64128     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
dense_1 (Dense)            

In [0]:
smaller_X_train = X_train[:TRAINING_SET_SIZE]

In [0]:
smaller_y_train = y_train[:TRAINING_SET_SIZE]

In [0]:
smaller_X_dev = X_test[:VAL_SET_SIZE]
smaller_y_dev = y_dev[:VAL_SET_SIZE]

In [0]:
# Run the model.
try:
  time_start = time.time()

  history = model.fit(X_train, y_train,
                      epochs=20,
                      verbose=True,
                      validation_data=(smaller_X_dev, smaller_y_dev),
                      batch_size=BATCH_SIZE)

  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))

except:
  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))  

Train on 3962284 samples, validate on 2000000 samples
Epoch 1/20
3962284/3962284 [==============================] - 122s 31us/sample - loss: 0.5187 - acc: 0.7639 - val_loss: 0.4367 - val_acc: 0.8414
Epoch 2/20
3962284/3962284 [==============================] - 123s 31us/sample - loss: 0.5029 - acc: 0.7712 - val_loss: 0.4353 - val_acc: 0.8442
Epoch 3/20
3962284/3962284 [==============================] - 124s 31us/sample - loss: 0.4979 - acc: 0.7735 - val_loss: 0.4375 - val_acc: 0.8426
Epoch 4/20
3962284/3962284 [==============================] - 121s 30us/sample - loss: 0.4946 - acc: 0.7749 - val_loss: 0.4463 - val_acc: 0.8378
Epoch 5/20
3962284/3962284 [==============================] - 117s 29us/sample - loss: 0.4923 - acc: 0.7759 - val_loss: 0.4425 - val_acc: 0.8372
Epoch 6/20
3962284/3962284 [==============================] - 115s 29us/sample - loss: 0.4905 - acc: 0.7767 - val_loss: 0.4364 - val_acc: 0.8420
Epoch 7/20
3962284/3962284 [==============================] - 114s 29us/samp

In [0]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_dev, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.7825
Testing Accuracy:  0.8338


In [0]:
# ROC AUC
from sklearn.metrics import roc_auc_score
# predict probabilities for test set
yhat_probs = model.predict(X_test, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:,0]
auc = roc_auc_score(y_dev, yhat_probs)
print('ROC AUC:', auc)


ROC AUC: 0.6204470364636516


In [0]:
# can't get this to work
# plot_model(history)

In [0]:
preds = yhat_probs # model.predict(X_test)

In [0]:
dev_df['probs'] = preds

In [0]:
def pred_to_label(row):
  if row['probs'] >= .5:
    return 'M'
  else:
    return 'W'

dev_df['preds'] = dev_df.apply(pred_to_label, axis=1)

In [0]:
wrong_preds = dev_df[dev_df.op_gender!=dev_df.preds]

In [0]:
wrong_preds.op_gender.value_counts()

W    278898
M     60440
Name: op_gender, dtype: int64

In [0]:
# total proportion of preds for women
dev_df.preds.value_counts()['W'] / len(dev_df)

0.04515253838864416

In [0]:
#random sample of mistakes
wrong_preds.head(20)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
2060436,11200190,48870736,W,394386,Grant,"aca raised my health care from $600 to $1,600 per month. it is anything but affordable. betty ...",Betty McCollum,Congress_Democratic,0.884766,M
1983886,11033278,86569077,W,389881,Deanna,i like the idea of paid fml but i would much rather see employers raise employees income overall...,Kirsten Gillibrand,Congress_Democratic,0.764868,M
1924717,10974109,86569077,W,389341,Raelee,thank you so much for committing to voting against betsy devos.,Kirsten Gillibrand,Congress_Democratic,0.765119,M
498563,4080762,77914417,M,142967,Gregg,"republicans have financial astuteness? bite your tongue, carla. can it be any more clear that th...",John Garamendi,Congress_Democratic,0.441456,W
53079,2170994,38794897,M,36862,Gail,proud of you senator brown. already voted for you.,Sherrod Brown,Congress_Democratic,0.235466,W
2037193,11086585,86569077,W,390472,Dwight,"mmmm lets see here donna karan, fashion designer and the creator of the donna karan new york and...",Kirsten Gillibrand,Congress_Democratic,0.809172,M
2037981,11087373,86569077,W,390484,Ron,"i have no problem with ending the subsidies, but i also do not mind paying more for all goods. g...",Kirsten Gillibrand,Congress_Democratic,0.838824,M
435979,3580654,84716153,M,122244,Donnie,"so senator, how much has she contributed to your campaign? $$$$",Jeff Flake,Congress_Republican,0.281930,W
2113460,11254440,77234050,W,399626,Debra,cecile charles i posted the table for you - it is for wa state,Cathy McMorris Rodgers,Congress_Republican,0.523769,M
2153417,11294397,77234050,W,400418,Hugh,former marine thanks you.,Cathy McMorris Rodgers,Congress_Republican,0.829357,M


In [0]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=False)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
2136861,11277841,77234050,W,399896,Carol,"please senator mcmorris, support rand paul in his efforts to protect our rights to due process",Cathy McMorris Rodgers,Congress_Republican,0.999905,M
2204895,12066357,87468720,W,449442,Brian,sen. david long has now officially endorsed lt. gov. eric holcomb to be our next indiana govenor...,Susan Brooks,Congress_Republican,0.999856,M
2148526,11289506,77234050,W,400233,Jeffrey,bravo congressman mcmorris-rodgers! keep up the good work! when will you return to the lc valley...,Cathy McMorris Rodgers,Congress_Republican,0.999826,M
2084297,11225277,77234050,W,399246,Elizabeth,happy father's day brian. you are truly a remarkable man that we are honored to know.,Cathy McMorris Rodgers,Congress_Republican,0.999726,M
2001084,11050476,86569077,W,390043,Margaret,"the shooter took his mother's gun. if these guns were still banned, she would not have it. mr ra...",Kirsten Gillibrand,Congress_Democratic,0.999697,M
2001453,11050845,86569077,W,390046,Bob,cram it up frat boy rand's a** he is no patriot.,Kirsten Gillibrand,Congress_Democratic,0.999666,M
2147711,11288691,77234050,W,400211,Gregory,"exactly, rep. mcmorris, that is exactly why we need the fairtax!!! no flat tax, the fairtax...f...",Cathy McMorris Rodgers,Congress_Republican,0.999653,M
389197,3139288,23984618,W,83580,Ali,rand paul 2016,Ann Wagner,Congress_Republican,0.999645,M
2065039,11204793,48870736,W,394961,Teresa,it has people like you rep. mccollum that are allowing a very un-popular president to put this c...,Betty McCollum,Congress_Democratic,0.999633,M
2160281,11301261,77234050,W,400760,John,i see your favorite heckler parke thomas still has not found something constructive to do with h...,Cathy McMorris Rodgers,Congress_Republican,0.999612,M


In [0]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=True)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
991634,5240688,46506154,M,161317,Laura,"keep the faith ms. ayotte, you are good for new hampshire and our country !! you have more supp...",Marco Rubio,Congress_Republican,0.000001,W
1887657,10920585,10458808,M,386074,Stephen,"is this actually going to be on the ballot. if so, ag kamala harris needs to take this kind of ...",Mark Takano,Congress_Democratic,0.000024,W
68491,2186406,38794897,M,37467,Sue,what is the status of your shared sacrifice bill introduced in april? i think it should be the ...,Sherrod Brown,Congress_Democratic,0.000034,W
1857510,10890438,10458808,M,385771,Kat,"thank you, congresswoman pelosi! 💖",Mark Takano,Congress_Democratic,0.000065,W
16846,2134761,38794897,M,36356,Laurie,i just found out that a lot of times you can be speaking to en empty room. that is something i ...,Sherrod Brown,Congress_Democratic,0.000069,W
1771059,10294196,32570461,M,382168,Cee,"waiting for my ballot. voting clinton/kaine, kamala harris and bryan caforio! straight blue!",Tim Kaine,Congress_Democratic,0.000082,W
1195246,6641430,48816171,M,252876,Norma,no marsha is getting it free practically because people like sonia have to pay for part of marsh...,Kenny Marchant,Congress_Republican,0.000095,W
887235,5136289,46506154,M,160981,Sven,"trash scott brown all you want, but he is infinitely better in the senate than jeanne shaheen. j...",Marco Rubio,Congress_Republican,0.000106,W
1887369,10920297,10458808,M,386070,Stephen,yes yes yes. kamala. you have always been there for us!!! fantastic!,Mark Takano,Congress_Democratic,0.000115,W
1373869,7792163,99469231,M,296202,Michael,as the terrorist attacks abroad continue....here at home (ignorance) on the behalf of state leve...,John McCain,Congress_Republican,0.000131,W
